In [54]:
import numpy as np
import yfinance as yf
import pandas as pd
import methods.selenium_patch as selenium_patch
from datetime import timedelta
from methods.scraper import Ticker

In [55]:
ticker = "AMKBY"
data = Ticker(ticker).macrotrends_key_financial_ratios(frequency="quarterly")
display(data)

field_name,Current Ratio,Long-term Debt / Capital,Debt/Equity Ratio,Gross Margin,Operating Margin,EBIT Margin,EBITDA Margin,Pre-Tax Profit Margin,Net Profit Margin,Asset Turnover,Inventory Turnover Ratio,Receiveable Turnover,Days Sales In Receivables,ROE - Return On Equity,Return On Tangible Equity,ROA - Return On Assets,ROI - Return On Investment,Book Value Per Share,Operating Cash Flow Per Share,Free Cash Flow Per Share
2025-09-30,2.15560,0.07570,0.10290,18.90750,8.12330,8.12330,,8.84130,7.37010,0.16010,7.91750,0.64000,140.61940,1.90490,2.32630,1.23520,1.76060,18.17460,0.8871,0.5118
2025-06-30,2.06120,0.06340,0.08800,17.50190,5.11810,5.11810,,5.59030,4.46310,0.14940,6.89500,0.59490,151.28640,1.11970,1.37250,0.72730,1.04870,18.02700,0.6189,0.2151
2025-03-31,2.14290,0.06380,0.09190,20.34380,8.59550,8.59550,,10.73490,8.72310,0.15320,7.24300,0.60220,149.45500,2.13800,2.58740,1.38790,2.00150,17.83300,0.88070,0.45310
2024-12-31,2.44720,0.07260,0.08740,24.50320,13.27260,13.27260,,15.44470,14.28670,0.16640,6.88190,0.60030,149.93630,3.64130,4.38460,2.40600,3.37680,18.30410,1.3975,0.8882
2024-09-30,2.52890,0.07770,0.09680,30.43400,20.57480,20.57480,,20.67000,19.34400,0.18560,7.33940,0.64950,138.56870,5.45340,6.61800,3.62720,5.02970,17.84630,1.3438,1.0818
2024-06-30,2.41040,0.08430,0.10160,16.78800,6.82010,6.82010,,7.64230,6.24850,0.15820,6.47990,0.65310,137.79420,1.56800,1.92780,1.03160,1.43580,16.78120,0.5292,0.3241
2024-03-31,2.43220,0.08870,0.10150,12.86930,0.63940,0.63940,,2.65480,1.43260,0.15140,6.54810,0.62100,144.91780,0.38970,0.47930,0.25490,0.35510,15.18910,0.34030,0.13450
2023-12-31,2.44770,0.07040,0.07930,6.56670,-5.32320,-5.32320,,-3.71350,-3.71350,0.14220,6.61640,0.58870,152.88730,-0.82770,-1.01410,-0.55220,-0.76950,15.67770,0.0378,-0.3169
2023-09-30,0.00250,0.06720,0.07540,15.48360,3.54520,3.54520,,5.69710,4.29550,145.32880,5794.79900,539.66630,0.16680,0.98980,-2.02100,663.79900,0.92330,15.92900,0.4526,0.2154
2023-06-30,2.55380,0.06120,0.06820,22.12040,11.27960,11.27960,,12.24980,11.18730,0.15550,6.82060,0.66340,135.65830,2.63530,3.22150,1.78080,2.47390,,0.8372,0.7073


In [56]:
prices = yf.Ticker(ticker).history("max").get("Close")
earning_prices = []
for date in data.index:
    got_price = False
    i = 0
    while got_price == False and i<7:
        try:
            earning_prices.append(prices[pd.to_datetime(date).tz_localize("America/New_York") - timedelta(days=i)]) # type: ignore
            got_price = True
        except:
            i += 1
    if got_price == False:
        earning_prices.append(np.nan)

earning_changes = [np.nan]
for i in range(1, len(earning_prices)):
    earning_changes.append((earning_prices[i-1]/earning_prices[i]-1)*100)

In [57]:
data["Future Change%"] = earning_changes
data["Price"] = earning_prices
data["Ticker"] = ticker
data = data[data.columns.to_list()[::-1]]
data

field_name,Ticker,Price,Future Change%,Free Cash Flow Per Share,Operating Cash Flow Per Share,Book Value Per Share,ROI - Return On Investment,ROA - Return On Assets,Return On Tangible Equity,ROE - Return On Equity,...,Asset Turnover,Net Profit Margin,Pre-Tax Profit Margin,EBITDA Margin,EBIT Margin,Operating Margin,Gross Margin,Debt/Equity Ratio,Long-term Debt / Capital,Current Ratio
2025-09-30,AMKBY,9.760000,NaN,0.5118,0.8871,18.17460,1.76060,1.23520,2.32630,1.90490,...,0.16010,7.37010,8.84130,,8.12330,8.12330,18.90750,0.10290,0.07570,2.15560
2025-06-30,AMKBY,9.230000,5.742153,0.2151,0.6189,18.02700,1.04870,0.72730,1.37250,1.11970,...,0.14940,4.46310,5.59030,,5.11810,5.11810,17.50190,0.08800,0.06340,2.06120
2025-03-31,AMKBY,8.680000,6.336397,0.45310,0.88070,17.83300,2.00150,1.38790,2.58740,2.13800,...,0.15320,8.72310,10.73490,,8.59550,8.59550,20.34380,0.09190,0.06380,2.14290
2024-12-31,AMKBY,7.522467,15.387680,0.8882,1.3975,18.30410,3.37680,2.40600,4.38460,3.64130,...,0.16640,14.28670,15.44470,,13.27260,13.27260,24.50320,0.08740,0.07260,2.44720
2024-09-30,AMKBY,7.650743,-1.676652,1.0818,1.3438,17.84630,5.02970,3.62720,6.61800,5.45340,...,0.18560,19.34400,20.67000,,20.57480,20.57480,30.43400,0.09680,0.07770,2.52890
2024-06-30,AMKBY,7.898132,-3.132245,0.3241,0.5292,16.78120,1.43580,1.03160,1.92780,1.56800,...,0.15820,6.24850,7.64230,,6.82010,6.82010,16.78800,0.10160,0.08430,2.41040
2024-03-31,AMKBY,5.732742,37.772324,0.13450,0.34030,15.18910,0.35510,0.25490,0.47930,0.38970,...,0.15140,1.43260,2.65480,,0.63940,0.63940,12.86930,0.10150,0.08870,2.43220
2023-12-31,AMKBY,7.454071,-23.092460,-0.3169,0.0378,15.67770,-0.76950,-0.55220,-1.01410,-0.82770,...,0.14220,-3.71350,-3.71350,,-5.32320,-5.32320,6.56670,0.07930,0.07040,2.44770
2023-09-30,AMKBY,7.487497,-0.446434,0.2154,0.4526,15.92900,0.92330,663.79900,-2.02100,0.98980,...,145.32880,4.29550,5.69710,,3.54520,3.54520,15.48360,0.07540,0.06720,0.00250
2023-06-30,AMKBY,7.312009,2.399997,0.7073,0.8372,,2.47390,1.78080,3.22150,2.63530,...,0.15550,11.18730,12.24980,,11.27960,11.27960,22.12040,0.06820,0.06120,2.55380


In [ ]:
def imputer(df: pd.DataFrame) -> pd.DataFrame:
    numerical_columns = df.columns.drop("Ticker")
    max_nans_share = 0.25
    row_nans = [0 for _ in df.index]
    col_nans = [0 for _ in numerical_columns]

    # Go through all data points and count nans
    i = 0
    for row in df.index:
        j = 0
        for column in numerical_columns:
            val = df.loc[row, column]
            if val == '':
                val = np.nan
                df.loc[row, column] = np.nan
            if np.isnan(float(val)): # type: ignore
                row_nans[i] += 1
                col_nans[j] += 1
            j += 1
        i += 1
                
    # Delete rows and columns with too many nans or missing price
    drop_rows = []
    for i in range(len(df.index)):
        if row_nans[i]/len(df.index) > max_nans_share or np.isnan(df.loc[df.index[i], "Price"]): # type: ignore
            drop_rows.append(df.index[i])
    drop_cols = []
    for j in range(len(numerical_columns)):
        if col_nans[j]/len(numerical_columns) > max_nans_share:
            drop_cols.append(numerical_columns[j])
    df = df.drop(drop_rows)
    df = df.drop(drop_cols, axis=1)

    return df

imputer(data)

field_name,Ticker,Price,Future Change%,Free Cash Flow Per Share,Operating Cash Flow Per Share,ROI - Return On Investment,ROA - Return On Assets,Return On Tangible Equity,ROE - Return On Equity,Days Sales In Receivables,...,Inventory Turnover Ratio,Asset Turnover,Net Profit Margin,Pre-Tax Profit Margin,EBIT Margin,Operating Margin,Gross Margin,Debt/Equity Ratio,Long-term Debt / Capital,Current Ratio
2025-09-30,AMKBY,9.760000,NaN,0.5118,0.8871,1.76060,1.23520,2.32630,1.90490,140.61940,...,7.91750,0.16010,7.37010,8.84130,8.12330,8.12330,18.90750,0.10290,0.07570,2.15560
2025-06-30,AMKBY,9.230000,5.742153,0.2151,0.6189,1.04870,0.72730,1.37250,1.11970,151.28640,...,6.89500,0.14940,4.46310,5.59030,5.11810,5.11810,17.50190,0.08800,0.06340,2.06120
2025-03-31,AMKBY,8.680000,6.336397,0.45310,0.88070,2.00150,1.38790,2.58740,2.13800,149.45500,...,7.24300,0.15320,8.72310,10.73490,8.59550,8.59550,20.34380,0.09190,0.06380,2.14290
2024-12-31,AMKBY,7.522467,15.387680,0.8882,1.3975,3.37680,2.40600,4.38460,3.64130,149.93630,...,6.88190,0.16640,14.28670,15.44470,13.27260,13.27260,24.50320,0.08740,0.07260,2.44720
2024-09-30,AMKBY,7.650743,-1.676652,1.0818,1.3438,5.02970,3.62720,6.61800,5.45340,138.56870,...,7.33940,0.18560,19.34400,20.67000,20.57480,20.57480,30.43400,0.09680,0.07770,2.52890
2024-06-30,AMKBY,7.898132,-3.132245,0.3241,0.5292,1.43580,1.03160,1.92780,1.56800,137.79420,...,6.47990,0.15820,6.24850,7.64230,6.82010,6.82010,16.78800,0.10160,0.08430,2.41040
2024-03-31,AMKBY,5.732742,37.772324,0.13450,0.34030,0.35510,0.25490,0.47930,0.38970,144.91780,...,6.54810,0.15140,1.43260,2.65480,0.63940,0.63940,12.86930,0.10150,0.08870,2.43220
2023-12-31,AMKBY,7.454071,-23.092460,-0.3169,0.0378,-0.76950,-0.55220,-1.01410,-0.82770,152.88730,...,6.61640,0.14220,-3.71350,-3.71350,-5.32320,-5.32320,6.56670,0.07930,0.07040,2.44770
2023-09-30,AMKBY,7.487497,-0.446434,0.2154,0.4526,0.92330,663.79900,-2.02100,0.98980,0.16680,...,5794.79900,145.32880,4.29550,5.69710,3.54520,3.54520,15.48360,0.07540,0.06720,0.00250
2023-06-30,AMKBY,7.312009,2.399997,0.7073,0.8372,2.47390,1.78080,3.22150,2.63530,135.65830,...,6.82060,0.15550,11.18730,12.24980,11.27960,11.27960,22.12040,0.06820,0.06120,2.55380
